# **Lunar Lander con Q-Learning**

### **1. Bibliotecas**

In [59]:
import sys
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random

### **2. Jugando a mano**

A continuación se puede jugar un episodio del lunar lander. Se controlan los motores con el teclado. Notar que solo se puede realizar una acción a la vez (que es parte del problema), y que en esta implementación, izq toma precedencia sobre derecha, que toma precedencia sobre el motor principal.

In [60]:
import pygame
from pygame.locals import *

# Inicializar pygame (para el contro|l con el teclado) y el ambiente
pygame.init()
env = gym.make('LunarLander-v2', render_mode='human')
env.reset()
pygame.display.set_caption('Lunar Lander')

clock = pygame.time.Clock()
done = False

while not done:
    for event in pygame.event.get():
        if event.type == QUIT:
            done = True
            break

    keys = pygame.key.get_pressed()

    # Map keys to actions
    if keys[K_LEFT]:
        action = 3  # Fire left orientation engine
    elif keys[K_RIGHT]:
        action = 1 # Fire right orientation engine
    elif keys[K_UP]:
        action = 2  # Fire main engine
    else:
        action = 0  # Do nothing

    _, _, terminated, truncated, _ = env.step(action)
    env.render()
    clock.tick(10)

    if terminated or truncated:
        done = True

env.close()
pygame.quit()

## **3. Discretizando el estado**

El estado consiste de posiciones y velocidades en (x,y,theta) y en información de contacto de los pies con la superficie.

Como varios de estos son continuos, tenemos que discretizarlos para aplicar nuestro algoritmo de aprendizaje por refuerzo tabular.

In [61]:
# Cuántos bins queremos por dimensión
# Pueden considerar variar este parámetro
bins_per_dim = 20

# Estado:
# (x, y, x_vel, y_vel, theta, theta_vel, pie_izq_en_contacto, pie_derecho_en_contacto)
NUM_BINS = [bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, 2, 2]

#env = gym.make('LunarLander-v2', render_mode='human')
#env.reset()

# Tomamos los rangos del env
OBS_SPACE_HIGH = env.observation_space.high
OBS_SPACE_LOW = env.observation_space.low
OBS_SPACE_LOW[1] = 0 # Para la coordenada y (altura), no podemos ir más abajo que la zona dea aterrizae (que está en el 0, 0)

# Los bins para cada dimensión
bins = [
    np.linspace(OBS_SPACE_LOW[i], OBS_SPACE_HIGH[i], NUM_BINS[i] - 1)
    for i in range(len(NUM_BINS) - 2) # last two are binary
]
# Se recomienda observar los bins para entender su estructura
#print ("Bins: ", bins)

def discretize_state(state, bins):
    """Discretize the continuous state into a tuple of discrete indices."""
    state_disc = list()
    for i in range(len(state)):
        if i >= len(bins):  # For binary features (leg contacts)
            state_disc.append(int(state[i]))
        else:
            state_disc.append(
                np.digitize(state[i], bins[i])
            )
    return tuple(state_disc)

In [62]:
# Ejemplos
print(discretize_state([0.0, 0.0, 0, 0, 0, 0, 1, 1], bins)) # En la zona de aterrizaje y quieto
print(discretize_state([0, 1.5, 0, 0, 0, 0, 0, 0], bins)) # Comenzando la partida, arriba y en el centro

(np.int64(10), np.int64(1), np.int64(10), np.int64(10), np.int64(10), np.int64(10), 1, 1)
(np.int64(10), np.int64(19), np.int64(10), np.int64(10), np.int64(10), np.int64(10), 0, 0)


## **4. Agentes y la interacción con el entorno**

Vamos a definir una interfaz para nuestro agente:

In [63]:
class Agente:
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        """Elegir la accion a tomar en el estado actual y el espacio de acciones
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó
        """
        pass

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        """Aprender a partir de la tupla 
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó en esta transición
        """
        pass

Para un agente aleatorio, la implementación sería:

In [64]:
import random

class AgenteAleatorio(Agente):
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        # Elige una acción al azar
        return random.randrange(max_accion)

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        # No aprende
        pass

Luego podemos definir una función para ejecutar un episodio con un agente dado:

In [65]:
def ejecutar_episodio(agente, aprender = True, render = None):
    entorno = gym.make('LunarLander-v2', render_mode=render).env

    iteraciones = 0
    recompensa_total = 0

    termino = False
    truncado = False
    estado_anterior, info = entorno.reset()
    while not termino and not truncado:
        # Le pedimos al agente que elija entre las posibles acciones (0..entorno.action_space.n)
        # Si no estamos aprendiendo, explotamos sin explorar
        accion = agente.elegir_accion(estado_anterior, entorno.action_space.n, not aprender)
        # Realizamos la accion
        estado_siguiente, recompensa, termino, truncado, info = entorno.step(accion)
        # Le informamos al agente para que aprenda
        if (aprender):
            agente.aprender(estado_anterior, estado_siguiente, accion, recompensa, termino)

        estado_anterior = estado_siguiente
        iteraciones += 1
        recompensa_total += recompensa
    env.close()
    return recompensa_total

In [66]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)

# Ejecutamos un episodio con el agente aleatorio y modo render 'human', para poder verlo
ejecutar_episodio(AgenteAleatorio(), render = 'human')

np.float64(-264.23108041898524)

Podemos ejecutar este ambiente muchas veces y tomar métricas al respecto

In [67]:
agente = AgenteAleatorio()
recompensa_episodios = []

exitos = 0
num_episodios = 100
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        exitos += 1
    recompensa_episodios += [recompensa]

import numpy
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

Tasa de éxito: 0.0. Se obtuvo -166.31271996915723 de recompensa, en promedio


### **5. Programando un agente que aprende**

La tarea a realizar consiste en programar un agente de aprendizaje por refuerzos:

In [68]:
import numpy as np
import random

class AgenteRL(Agente):
    def __init__(self, bins, num_acciones, alpha=1, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        # Inicializa los parámetros del agente
        super().__init__()
        self.alpha = alpha  # Tasa de aprendizaje
        self.gamma = gamma  # Factor de descuento
        self.epsilon = epsilon  # Factor de exploración
        self.epsilon_min = epsilon_min  # Valor mínimo de epsilon para exploración
        self.epsilon_decay = epsilon_decay  # Tasa de decaimiento de epsilon
        self.bins = bins  # Bins para discretizar el estado
        self.q_table = np.zeros((*[len(b) + 1 for b in bins], 2, 2, num_acciones))  # Q-table con dimensiones basadas en bins y los estados binarios


    def elegir_accion(self, estado,max_accion, explorar=True) -> int:
        # Discretizamos el estado antes de elegir una acción
        estado_discreto = discretize_state(estado,self.bins)
        
        # Política epsilon-greedy para elegir acción
        if explorar and np.random.rand() < self.epsilon:
            # Explorar: elige una acción aleatoria
            return np.random.randint(max_accion)
        else:
            # Explotar: elige la acción con el mayor valor Q para el estado actual
            return np.argmax(self.q_table[estado_discreto])

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        # Discretizamos los estados antes de actualizar la Q-table
        estado_anterior_discreto = discretize_state(estado_anterior,self.bins)
        estado_siguiente_discreto = discretize_state(estado_siguiente,self.bins)
        
        # Actualiza la Q-table usando la ecuación de Q-Learning
        max_accion_siguiente = np.max(self.q_table[estado_siguiente_discreto])  # Mejor acción posible en el siguiente estado
        q_actual = self.q_table[estado_anterior_discreto + (accion,)]  # Valor actual de Q para el estado y la acción
        
        # Si el episodio terminó, no hay valor futuro, de lo contrario aplicamos la ecuación de Bellman
        if terminado:
            q_nuevo = recompensa  # Si terminó, la recompensa es lo único que importa
        else:
            q_nuevo = recompensa + self.gamma * max_accion_siguiente  # Bellman update

        # Actualizamos la Q-table con el nuevo valor Q
        self.q_table[estado_anterior_discreto + (accion,)] += self.alpha * (q_nuevo - q_actual)

        # Reducimos epsilon para que exploremos me|nos con el tiempo (decaimiento)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def fin_episodio(self):
        # Función que se llama al final de cada episodio para realizar tareas adicionales si es necesario
        pass


Y ejecutar con el muchos episodios:

In [69]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)
# Advertencia: este bloque es un loop infinito si el agente se deja sin implementar

entorno = gym.make('LunarLander-v2').env
agente = AgenteRL(bins=bins, num_acciones=entorno.action_space.n)
exitos = 0
recompensa_episodios = []
num_episodios = 1000
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        print(f"Episodio {i} exitoso, recompensa: {recompensa}")
        exitos += 1
    recompensa_episodios += [recompensa]
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

Episodio 1378 exitoso, recompensa: 261.5772210770633
Episodio 1464 exitoso, recompensa: 235.08042903777573
Episodio 1725 exitoso, recompensa: 221.74124688441404
Episodio 1832 exitoso, recompensa: 248.70131582428598
Episodio 2273 exitoso, recompensa: 244.61464706448893
Episodio 2389 exitoso, recompensa: 214.54473683760403


KeyboardInterrupt: 

In [58]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)
# Advertencia: este bloque es un loop infinito si el agente se deja sin implementar

entorno = gym.make('LunarLander-v2').env
exitos = 0
recompensa_episodios = []
num_episodios = 1000
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente,aprender=False)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        print(f"Episodio {i} exitoso, recompensa: {recompensa}")
        exitos += 1
    recompensa_episodios += [recompensa]
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

Tasa de éxito: 0.0. Se obtuvo -191.7810841285604 de recompensa, en promedio


Analizar los resultados de la ejecución anterior, incluyendo:
 * Un análisis de los parámetros utilizados en el algoritmo (aprendizaje, política de exploración)
 * Un análisis de algunos 'cortes' de la matriz Q y la política (p.e. qué hace la nave cuando está cayendo rápidamente hacia abajo, sin rotación)
 * Un análisis de la evolución de la recompensa promedio
 * Un análisis de los casos de éxito
 * Un análisis de los casos en el que el agente falla
 * Qué limitante del agente de RL les parece que afecta más negativamente su desempeño. Cómo lo mejorarían? 

In [10]:
# Analizar los resultados aqui
